# init

In [2]:
import numpy as np
import pandas as pd
import polars as pl
from timeit import timeit
import os, sys
print(os.getcwd())
os.chdir('/gpfs/data/healthcare-allocate/CLIF-MIMIC/code')
print(os.getcwd())

proj_root = "/gpfs/data/healthcare-allocate/CLIF-MIMIC"
if proj_root not in sys.path:
    sys.path.append(proj_root)

from code.custom_utils import *

/gpfs/data/healthcare-allocate/CLIF-MIMIC/code
/gpfs/data/healthcare-allocate/CLIF-MIMIC/code


In [3]:
def load_mimic_table(module, table, file_type):
    if file_type in ["pq", "parquet"]:
        return pd.read_parquet(f'../mimic-iv-2.2/{module}/{table}.parquet')
    elif file_type == "csv":
        return pd.read_csv(f'../mimic-iv-2.2/{module}/{table}.csv.gz')

d_items = load_mimic_table("icu", "d_items", "csv")
chartevents = load_mimic_table("icu", "chartevents", "parquet")
procedureevents = load_mimic_table("icu", "procedureevents", "csv")
datetimeevents = load_mimic_table("icu", "datetimeevents", "csv")
inputevents = load_mimic_table("icu", "inputevents", "csv")
outputevents = load_mimic_table("icu", "outputevents", "csv")
# d_items = pd.read_csv("../mimic-iv-2.2/icu/d_items.csv.gz")
# chartevents = pd.read_parquet('../mimic-iv-2.2/icu/chartevents.parquet')

In [17]:
outputevents = load_mimic_table("icu", "outputevents", "csv")

# utils

## `ItemFinder`

In [12]:
def item_id_to_events(item_id: int, df = chartevents):
    '''
    Return all the events associated with an item id in a pandas df.
    '''
    return df.loc[df["itemid"] == item_id, :]

class ItemFinder():
    def __init__(self, kw, df = d_items, col: str = "label", 
                 case: bool = False):
        '''
        Look up an item by keyword from the `d_items` table of the `icu` module.
        - df = `d_items`
        - col = {"label", "abbr"}
        '''
        self.kw = kw 
        self.df = df
        self.col = "abbreviation" if col == "abbr" else col
        self.case = case

        # df of items that match the key words
        self.kw_items_df = df[
            df[self.col].str.contains(kw, case = case, na = False)
        ]
        # list of ids for items that match the key words
        self.kw_items_ids = self.kw_items_df["itemid"].values
        # a string list of non-duplicated events table names, e.g. ["chartevents", "procedureevents"]
        self.linksto_table_names = self.kw_items_df["linksto"].unique()
                                                                
        self.kw_chartevents = chartevents.loc[
            chartevents["itemid"].isin(self.kw_items_ids),
            :
        ]
        self.item_freq = self.generate_item_freq()

        self.candidate_table = self.make_candidate_table()

    def generate_item_freq(self):
        '''
        Iterative over each events table, find the items freq therein, and combine into one df.
        '''
        freq_df_ls = [] # a list of df's
        for table_name in self.linksto_table_names:
            # fetch the object by name, i.e. chartevents, procedureevents, etc.
            events_df = globals()[table_name]
            # a df of events associated with the key word items
            kw_events_df = events_df.loc[
                events_df["itemid"].isin(self.kw_items_ids),
                :
            ]
             
            # a df of item freq for one event type  
            item_freq_df = kw_events_df.value_counts("itemid")

            # check if the df is empty
            if not item_freq_df.empty:
                freq_df_ls.append(item_freq_df)
        
        # check if the list len is 0, meaning 
        if len(freq_df_ls) != 0:
            # return a df of all the item freq of all events table concat-ed together
            return pd.concat(freq_df_ls)
        else: 
            return self.kw_chartevents.value_counts("itemid")

    def make_candidate_table(self):
        '''
        # TODO
        '''
        if not self.item_freq.empty:
            return (
                self.kw_items_df
                .loc[:, ["itemid", "label", "abbreviation", "linksto", "category", "unitname", "param_type"]]
                # FIXME
                .join(self.item_freq, on = "itemid", validate = "1:1")
                .sort_values(by = "count", ascending = False) 
                .assign(
                    value_instances = lambda x: x["itemid"].apply(ItemFinder.item_id_to_value_instances)
                )
            )
        else: 
            return "No matching result found."
        
    @staticmethod
    def item_id_to_label(item_id: int) -> str:
        '''
        Helper function that returns the "label" string of an item given its item_id. 
        '''
        return d_items.loc[d_items["itemid"] == item_id, "label"].values[0]

    @staticmethod
    def item_id_to_value_instances(item_id: int):
        '''
        Wrapper
        '''
        label = ItemFinder.item_id_to_label(item_id)

        param_type = d_items.loc[d_items["itemid"] == item_id, "param_type"].values[0]
        if param_type == "Numeric":
            val_instances = ItemFinder.item_id_to_value_instances_numeric(item_id)
        elif param_type == "Text":
            val_instances = ItemFinder.item_id_to_value_instances_categorical(item_id)
        else:
            return param_type
        print(f"item label: {label}; value instances: {str(val_instances)}")
        return val_instances

    @staticmethod
    def item_id_to_value_instances_categorical(item_id: int):
        '''
        Return all the unique categories
        '''
        assoc_chartevents = chartevents.loc[chartevents["itemid"] == item_id, :]
        categories = assoc_chartevents.value_counts("value").index.to_list()
        return str(categories)
    
    @staticmethod
    def item_id_to_value_instances_numeric(item_id: int):
        '''
        Find max, min, mean of a continuous, or numeric, item.
        '''
        valuenum_col = chartevents.loc[chartevents["itemid"] == item_id, :]["valuenum"]
        val_max, val_min, val_mean = valuenum_col.max(), valuenum_col.min(), valuenum_col.mean()
        return f"Max: {val_max}, Min: {val_min}, Mean: {val_mean}"

In [9]:
ItemFinder("Temperature").candidate_table

item label is Temperature Site
categories are ['Oral', 'Blood', 'Axillary', 'Rectal', 'Esophogeal', 'Temporal', 'Tympanic']
item label is Temperature Fahrenheit
item label is Skin Temperature
categories are ['Warm', 'Cool', 'Hot', 'Cold']
item label is Temperature Celsius
item label is Blood Temperature CCO (C)
item label is Cerebral Temperature (C)
item label is Changes in Temperature
categories are ['No report of temperature change', 'Reports feeling cold, hands cold and clammy to touch', 'Uncontrolled shivering']
item label is TemperatureF_ApacheIV
item label is Pt. Temperature (BG) (SOFT)


,itemid,label,abbreviation,category,unitname,param_type,count,value_instances
767,224642,Temperature Site,Temp Site,Routine Vital Signs,NaN,Text,1842387.0,"['Oral', 'Blood', 'Axillary', 'Rectal', 'Esoph..."
337,223761,Temperature Fahrenheit,Temperature F,Routine Vital Signs,°F,Numeric,1515962.0,"Max: 234123.0, Min: -99.9, Mean: 98.7157890197..."
505,224027,Skin Temperature,Skin Temp,Skin - Assessment,NaN,Text,952619.0,"['Warm', 'Cool', 'Hot', 'Cold']"
338,223762,Temperature Celsius,Temperature C,Routine Vital Signs,°C,Numeric,264628.0,"Max: 987.4, Min: 0.0, Mean: 37.05203024169777"
1814,226329,Blood Temperature CCO (C),Blood Temp CCO (C),Routine Vital Signs,°C,Numeric,151805.0,"Max: 336.7, Min: 0.0, Mean: 37.07423083165904"
3466,229236,Cerebral Temperature (C),Cerebral T (C),Hemodynamics,°C,Numeric,2469.0,"Max: 39.47, Min: 32.9, Mean: 37.076647225597405"
790,224674,Changes in Temperature,Changes in Temperature,Toxicology,NaN,Text,1972.0,"['No report of temperature change', 'Reports f..."
2097,227054,TemperatureF_ApacheIV,TemperatureF_ApacheIV,Scores - APACHE IV (2),°F,Numeric,7.0,"Max: 103.6, Min: 96.0, Mean: 99.8"
2776,228242,Pt. Temperature (BG) (SOFT),Pt. Temperature (BG) (SOFT),Labs,NaN,Numeric,NaN,"Max: nan, Min: nan, Mean: nan"


# by MIMIC modules / tables

## `hosp`

In [2]:
d_labitems = pd.read_csv("mimic-iv-2.2/hosp/d_labitems.csv.gz")

In [4]:
# d_labitems.head(10)
contains = d_labitems["label"].str.contains("temp", case=False, na=False)
d_labitems[contains]

,itemid,label,fluid,category
23,50825,Temperature,Blood,Blood Gas


In [19]:
d_labitems["fluid"].unique()
d_labitems["category"].unique()

array(['Blood Gas', 'Chemistry', 'Hematology'], dtype=object)

In [12]:
omr = pd.read_csv("../mimic-iv-2.2/hosp/omr.csv.gz")
omr.head(10)

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15
5,10000032,2180-05-07,2,Weight (Lbs),92.15
6,10000032,2180-05-07,3,Weight (Lbs),92.15
7,10000032,2180-05-07,4,Weight (Lbs),92.15
8,10000032,2180-05-07,5,Weight (Lbs),92.15
9,10000032,2180-05-07,6,Weight (Lbs),92.15


## `icu`

### `d_items`

### `chartevents`

In [3]:
# resave into parquet
chartevents = load_gz_csv("mimic-iv-2.2/icu/chartevents.csv.gz")
chartevents.to_parquet('mimic-iv-2.2/icu/chartevents.parquet')  

In [54]:
chartevents.loc[
    chartevents["itemid"] == 224689, :]["valuenum"].agg(["max","min"])

max    2843.0
min       0.0
Name: valuenum, dtype: float64

In [16]:
items_category_freq = d_items.value_counts("category")

In [ ]:
def map_one_item(item_id):
    '''
    create the rows in a vitals table corresponding to one item from MIMIC, based on item id.
    '''


In [47]:
item_id = 220050
vital_name = ItemFinder.find_item_label(item_id)
vital_category = "sbp"
meas_site_name = ItemFinder.itemid_to_meas_site_name[item_id]

vitals_sbp = (
    chartevents
    .loc[
        chartevents["itemid"] == item_id,  # :
        ["hadm_id", "charttime", "itemid", "value"]
    ]
    .rename(columns = {
        "hadm_id": "encounter_id", 
        "charttime": "recorded_dttm",
        "value": "vital_value"
        })
    .assign(
        vital_name = vital_name,
        vital_category = vital_category,
        meas_site_name = meas_site_name
    )
    .reindex(
        columns = ["encounter_id", "recorded_dttm", "vital_name", "vital_category", "vital_value", "meas_site_name"]
    )
)


In [48]:
vitals_sbp

,encounter_id,recorded_dttm,vital_name,vital_category,vital_value,meas_site_name
19182,23581541,2160-05-18 20:00:00,Arterial Blood Pressure systolic,sbp,116,arterial
19327,23581541,2160-05-18 21:00:00,Arterial Blood Pressure systolic,sbp,97,arterial
19353,23581541,2160-05-18 22:00:00,Arterial Blood Pressure systolic,sbp,123,arterial
19376,23581541,2160-05-18 22:10:00,Arterial Blood Pressure systolic,sbp,94,arterial
19385,23581541,2160-05-18 23:00:00,Arterial Blood Pressure systolic,sbp,116,arterial
...,...,...,...,...,...,...
313630748,26785317,2148-11-20 06:00:00,Arterial Blood Pressure systolic,sbp,124,arterial
313630778,26785317,2148-11-20 20:00:00,Arterial Blood Pressure systolic,sbp,104,arterial
313630874,26785317,2148-11-20 21:00:00,Arterial Blood Pressure systolic,sbp,102,arterial
313630901,26785317,2148-11-20 22:00:00,Arterial Blood Pressure systolic,sbp,105,arterial


### `procedureevents`

In [22]:
item_id_to_events()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10000032,29079034,39553978,88981.0,2180-07-23 14:43:00,2180-07-23 14:44:00,2180-07-23 14:43:00,225966,1.000000,NaN,...,6416557,6416557,Procedures,Task,39.4,0,0,FinishedRunning,1.0,0
1,10000032,29079034,39553978,NaN,2180-07-23 14:24:00,2180-07-23 23:50:00,2180-07-23 23:50:49.983,224275,566.000000,min,...,6497934,6497934,Peripheral Lines,ContinuousProcess,39.4,1,0,FinishedRunning,566.0,1
2,10000032,29079034,39553978,NaN,2180-07-23 14:24:00,2180-07-23 23:50:00,2180-07-23 23:50:49.983,224277,566.000000,min,...,9643097,9643097,Peripheral Lines,ContinuousProcess,39.4,1,0,FinishedRunning,566.0,1
3,10000980,26913865,39765666,NaN,2189-06-27 09:01:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,225794,697.000000,min,...,5989583,5989583,Ventilation,ContinuousProcess,76.2,1,0,FinishedRunning,697.0,1
4,10000980,26913865,39765666,NaN,2189-06-27 09:15:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,224277,683.000000,min,...,476764,476764,Peripheral Lines,ContinuousProcess,76.2,1,0,FinishedRunning,683.0,1
5,10000980,26913865,39765666,NaN,2189-06-27 09:16:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,224275,682.000000,min,...,3149795,3149795,Peripheral Lines,ContinuousProcess,76.2,1,0,FinishedRunning,682.0,1
6,10001217,24597018,37067082,7355.0,2157-11-21 20:30:00,2157-11-21 20:31:00,2157-11-21 21:36:00,228129,1.000000,NaN,...,3280088,3280088,Communication,Task,71.2,0,0,FinishedRunning,1.0,0
7,10001217,24597018,37067082,7355.0,2157-11-21 20:30:00,2157-11-21 22:06:00,2157-11-21 21:35:00,224274,0.066667,day,...,3448258,3448258,Peripheral Lines,ContinuousProcess,71.2,0,0,Stopped,4.0,1
8,10001217,24597018,37067082,72136.0,2157-11-21 14:00:00,2157-11-21 14:01:00,2157-11-21 15:45:00,221214,1.000000,NaN,...,3969862,3969862,Imaging,Task,71.2,0,0,FinishedRunning,1.0,0
9,10001217,27703517,34592300,18215.0,2157-12-19 20:19:00,2157-12-19 20:20:00,2157-12-19 21:19:00,221214,1.000000,NaN,...,2956175,2956175,Imaging,Task,74.8,0,0,FinishedRunning,1.0,0


# by CLIF tables

## `vitals`

### Routine Vital Signs

In [12]:
routine_vitals = ItemFinder("Routine Vital Signs", col = "category").candidate_table
routine_vitals

item label is Heart Rate
item label is Heart Rhythm
categories are ['SR (Sinus Rhythm)', 'ST (Sinus Tachycardia) ', 'AF (Atrial Fibrillation)', 'SB (Sinus Bradycardia)', 'V Paced', '1st AV (First degree AV Block) ', 'A Paced', 'AV Paced', 'A Flut (Atrial Flutter) ', 'RBBB (Right Bundle Branch Block) ', 'SA (Sinus Arrhythmia)', 'LBBB (Left Bundle Branch Block) ', 'JR (Junctional Rhythm)', '3rd AV (Complete Heart Block) ', 'SVT (Supra Ventricular Tachycardia)', '2nd AV W-M1 (Second degree AV Block Wenckebach - Mobitz1) ', 'JT (Junctional Tachycardia) ', 'WAP (Wandering atrial pacemaker)', '2nd AV M2 (Second degree AV Block - Mobitz 2) ', 'MAT (Multifocal atrial tachycardia)', 'VT (Ventricular Tachycardia) ', 'PAT (Paroxysmal Atrial Tachycardia)', 'Asystole', 'Idioventricular', 'VF (Ventricular Fibrillation) ', 'PJT (Paroxysmal Junctional Tachycardia) ']
item label is Ectopy Type 1
categories are ["PVC's", "PAC's", 'Vent. Bigeminy', 'Atrial Bigeminy', 'Vent. Trigeminy', 'Nod/Junc Escape',

,itemid,label,abbreviation,category,unitname,param_type,count,value_instances
2,220045,Heart Rate,HR,Routine Vital Signs,bpm,Numeric,6460839,"Max: 10000000.0, Min: -241395.0, Mean: 87.8015..."
5,220048,Heart Rhythm,Heart Rhythm,Routine Vital Signs,NaN,Text,5907987,"['SR (Sinus Rhythm)', 'ST (Sinus Tachycardia) ..."
772,224650,Ectopy Type 1,Ectopy Type 1,Routine Vital Signs,NaN,Text,5272404,"[""PVC's"", ""PAC's"", 'Vent. Bigeminy', 'Atrial B..."
24,220179,Non Invasive Blood Pressure systolic,NBPs,Routine Vital Signs,mmHg,Numeric,4067219,"Max: 1025100.0, Min: -69.0, Mean: 120.67245107..."
25,220180,Non Invasive Blood Pressure diastolic,NBPd,Routine Vital Signs,mmHg,Numeric,4066337,"Max: 114109.0, Min: -2.0, Mean: 66.418738289522"
26,220181,Non Invasive Blood Pressure mean,NBPm,Routine Vital Signs,mmHg,Numeric,4064537,"Max: 8888890.0, Min: -22767.0, Mean: 84.590976..."
8,220052,Arterial Blood Pressure mean,ABPm,Routine Vital Signs,mmHg,Numeric,2272013,"Max: 930000.0, Min: -135.0, Mean: 80.147497615..."
6,220050,Arterial Blood Pressure systolic,ABPs,Routine Vital Signs,mmHg,Numeric,2264001,"Max: 95119.0, Min: -94.0, Mean: 119.3417890414..."
7,220051,Arterial Blood Pressure diastolic,ABPd,Routine Vital Signs,mmHg,Numeric,2263647,"Max: 114100.0, Min: -40.0, Mean: 60.0630160091..."
767,224642,Temperature Site,Temp Site,Routine Vital Signs,NaN,Text,1842387,"['Oral', 'Blood', 'Axillary', 'Rectal', 'Esoph..."


### `respiratory_rate`

In [62]:
ItemFinder("respiratory rate").selected_items

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
28,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
799,224688,Respiratory Rate (Set),Respiratory Rate (Set),chartevents,Respiratory,insp/min,Numeric,NaN,NaN
800,224689,Respiratory Rate (spontaneous),Respiratory Rate (spontaneous),chartevents,Respiratory,insp/min,Numeric,NaN,NaN
801,224690,Respiratory Rate (Total),Respiratory Rate (Total),chartevents,Respiratory,insp/min,Numeric,NaN,36.0


### `temp_c`

In [37]:
# temp site
item_id_to_events(224642)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
18,10000032,29079034,39553978,66056.0,2180-07-23 20:00:00,2180-07-23 19:59:00,224642,Oral,NaN,None,0.0
160,10000032,29079034,39553978,88981.0,2180-07-23 14:00:00,2180-07-23 14:18:00,224642,Oral,NaN,None,0.0
370,10000032,29079034,39553978,88981.0,2180-07-23 17:00:00,2180-07-23 17:03:00,224642,Oral,NaN,None,0.0
551,10000980,26913865,39765666,36518.0,2189-06-27 09:07:00,2189-06-27 09:07:00,224642,Axillary,NaN,None,0.0
733,10000980,26913865,39765666,36518.0,2189-06-27 12:00:00,2189-06-27 12:53:00,224642,Oral,NaN,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313644553,19999987,23865745,36195440,86756.0,2145-11-03 04:00:00,2145-11-03 04:24:00,224642,Oral,NaN,None,0.0
313644735,19999987,23865745,36195440,90295.0,2145-11-02 23:24:00,2145-11-02 23:24:00,224642,Oral,NaN,None,0.0
313644811,19999987,23865745,36195440,90295.0,2145-11-02 23:41:00,2145-11-02 23:41:00,224642,Oral,NaN,None,0.0
313644876,19999987,23865745,36195440,91879.0,2145-11-04 20:00:00,2145-11-04 19:38:00,224642,Oral,NaN,None,0.0


#### temp site

In [39]:
# levels of temp_site
item_id_to_events(224642).value_counts("value")

value
Oral          1278499
Blood          269153
Axillary       157738
Rectal          50577
Esophogeal      43733
Temporal        39990
Tympanic         2697
Name: count, dtype: int64

In [ ]:
temp_f_events = item_id_to_events(223761)
temp_c_events = item_id_to_events(223762)
temp_site_events = item_id_to_events(224642)

shared_stay_ids = set.intersection(
    set(temp_c_events["stay_id"].unique()),
    set(temp_site_events["stay_id"].unique())
)

shared_stay_ids

7048

In [ ]:
sid = 33851436
f_events_slc = temp_f_events.loc[
    temp_f_events["stay_id"] == sid, :
]
site_events_slc = temp_site_events.loc[
    temp_site_events["stay_id"] == sid, :
]

### `map`

In [ ]:
ItemFinder("map", col="abbr").candidate_table

item label is IABP Mean
item label is MapApacheIIScore
item label is PotassiumApacheIIScore
item label is SodiumApacheIIScore
item label is MapApacheIIValue
item label is PotassiumApacheIIValue
item label is SodiumApacheIIValue
item label is MapScore_ApacheIV
item label is MAP_ApacheIV


,itemid,label,abbreviation,category,unitname,param_type,count,value_instances
635,224322,IABP Mean,MAP - Assisted,IABP,mmHg,Numeric,31605,"Max: 701.0, Min: 0.0, Mean: 80.09030216737858"
1975,226765,MapApacheIIScore,MapApacheIIScore,Scores - APACHE II,NaN,Numeric,19,"Max: 4.0, Min: 0.0, Mean: 1.7894736842105263"
1981,226771,PotassiumApacheIIScore,PotassiumApacheIIScore,Scores - APACHE II,NaN,Numeric,19,"Max: 1.0, Min: 0.0, Mean: 0.2631578947368421"
1985,226775,SodiumApacheIIScore,SodiumApacheIIScore,Scores - APACHE II,NaN,Numeric,19,"Max: 1.0, Min: 0.0, Mean: 0.10526315789473684"
1976,226766,MapApacheIIValue,MapApacheIIValue,Scores - APACHE II,mmHg,Numeric,18,"Max: 124.0, Min: 25.0, Mean: 62.611111111111114"
1982,226772,PotassiumApacheIIValue,PotassiumApacheIIValue,Scores - APACHE II,NaN,Numeric,17,"Max: 5.0, Min: 3.0, Mean: 3.9647058823529413"
1986,226776,SodiumApacheIIValue,SodiumApacheIIValue,Scores - APACHE II,NaN,Numeric,17,"Max: 150.0, Min: 136.0, Mean: 142.05882352941177"
2067,227024,MapScore_ApacheIV,MapScore_ApacheIV,Scores - APACHE IV (2),NaN,Numeric,8,"Max: 15.0, Min: 0.0, Mean: 7.75"
2066,227023,MAP_ApacheIV,MAP_ApacheIV,Scores - APACHE IV (2),mmHg,Numeric,7,"Max: 122.0, Min: 54.0, Mean: 73.85714285714286"


In [ ]:
ItemFinder("Mean art").candidate_table

,itemid,label,abbreviation,category,unitname,param_type,count,value_instances


### sbp

In [ ]:
ItemFinder("systolic").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
24,220179,Non Invasive Blood Pressure systolic,NBPs,Routine Vital Signs,mmHg,Numeric,4067219.0
6,220050,Arterial Blood Pressure systolic,ABPs,Routine Vital Signs,mmHg,Numeric,2264001.0
11,220059,Pulmonary Artery Pressure systolic,PAPs,Hemodynamics,mmHg,Numeric,309026.0
1212,225309,ART BP Systolic,ART BP Systolic,Routine Vital Signs,mmHg,Numeric,228488.0
2694,228152,Aortic Pressure Signal - Systolic,Aortic Pressure Signal - Systolic,Impella,mmHg.,Numeric,8684.0
575,224167,Manual Blood Pressure Systolic Left,Manual BPs L,Routine Vital Signs,mmHg,Numeric,1400.0
2155,227243,Manual Blood Pressure Systolic Right,Manual BPs R,Routine Vital Signs,mmHg,Numeric,1136.0
3778,229669,Pulmonary Atrtery Pressure Signal - Systolic,Pulmonary Artery Pressure Signal - Systolic,Impella,mmHg.,Numeric,848.0
1995,226850,RV systolic pressure(PA Line),RV systolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN
1997,226852,PA systolic pressure(PA Line),PA systolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN


### dbp

In [ ]:
ItemFinder("diastolic").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
25,220180,Non Invasive Blood Pressure diastolic,NBPd,Routine Vital Signs,mmHg,Numeric,4066337.0
7,220051,Arterial Blood Pressure diastolic,ABPd,Routine Vital Signs,mmHg,Numeric,2263647.0
12,220060,Pulmonary Artery Pressure diastolic,PAPd,Hemodynamics,mmHg,Numeric,309126.0
1213,225310,ART BP Diastolic,ART BP Diastolic,Routine Vital Signs,mmHg,Numeric,228417.0
2693,228151,Aortic Pressure Signal - Diastolic,Aortic Pressure Signal - Diastolic,Impella,mmHg.,Numeric,8678.0
768,224643,Manual Blood Pressure Diastolic Left,Manual BPd L,Routine Vital Signs,mmHg,Numeric,1296.0
2154,227242,Manual Blood Pressure Diastolic Right,Manual BPd R,Routine Vital Signs,mmHg,Numeric,1042.0
3777,229668,Pulmonary Artery Pressure Signal - Diastolic,Pulmonary Artery Pressure Signal - Diastolic,Impella,mmHg.,Numeric,849.0
1996,226851,RV diastolic pressure(PA Line),RV diastolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN
1998,226853,PA diastolic pressure(PA Line),PA diastolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN


### `pulse`

In [ ]:
ItemFinder('pulse', col="abbr").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
430,223934,Dorsal PedPulse R,Dorsal PedPulse R,Cardiovascular (Pulses),NaN,Text,1027492
438,223943,Dorsal PedPulse L,Dorsal PedPulse L,Cardiovascular (Pulses),NaN,Text,1013331
431,223935,PostTib. Pulses R,PostTib Pulses R,Cardiovascular (Pulses),NaN,Text,757423
442,223947,PostTib. Pulses L,PostTib Pulses L,Cardiovascular (Pulses),NaN,Text,744884
432,223936,Radial Pulse R,Radial Pulse R,Cardiovascular (Pulses),NaN,Text,514582
443,223948,Radial Pulse L,Radial Pulse L,Cardiovascular (Pulses),NaN,Text,505331
433,223938,Ulnar Pulse R,Ulnar Pulse R,Cardiovascular (Pulses),NaN,Text,20949
444,223949,Ulnar Pulse L,Ulnar Pulse L,Cardiovascular (Pulses),NaN,Text,18107
435,223940,Femoral Pulse R,Femoral Pulse R,Cardiovascular (Pulses),NaN,Text,13552
434,223939,Brachial Pulse R,Brachial Pulse R,Cardiovascular (Pulses),NaN,Text,12171


In [ ]:
ItemFinder('heart rate').create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
2,220045,Heart Rate,HR,Routine Vital Signs,bpm,Numeric,6460839
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,Alarms,bpm,Numeric,627396
3,220046,Heart rate Alarm - High,HR Alarm - High,Alarms,bpm,Numeric,627111


In [ ]:
find_value_instances(223934)

item label is Dorsal PedPulse R


Index(['Easily Palpable', 'Doppler', 'Weak Palpable', 'Strong/Palpable',
       'Difficult to Palpate', 'Absent', 'Not Applicable'],
      dtype='object', name='value')

In [ ]:
ItemFinder.find_value_instances(229770)

item label is Resting Pulse Rate (COWS)


Index(['Pulse rate 80 or below', 'Pulse rate 81-100', 'Pulse rate 101-120',
       'Pulse rate greater than 120'],
      dtype='object', name='value')

### spo2

In [ ]:
ItemFinder('spo2', col = "abbr").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
36,220277,O2 saturation pulseoxymetry,SpO2,Respiratory,%,Numeric,6324341
346,223770,O2 Saturation Pulseoxymetry Alarm - Low,SpO2 Alarm - Low,Alarms,%,Numeric,622764
345,223769,O2 Saturation Pulseoxymetry Alarm - High,SpO2 Alarm - High,Alarms,%,Numeric,619601
1810,226253,SpO2 Desat Limit,SpO2 Desat Limit,Alarms,%,Numeric,600821
3920,229862,Forehead SpO2 Sensor in Place,Forehead SpO2 Sensor in Place,Routine Vital Signs,NaN,Checkbox,35


### `height_inches`

In [ ]:
ItemFinder("height").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
1941,226707,Height,Height,General,Inch,Numeric,33707
1945,226730,Height (cm),Height (cm),General,cm,Numeric,33707


## `labs`

In [ ]:
ItemFinder("protein").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
2205,227444,C Reactive Protein (CRP),C Reactive Protein (CRP),Labs,NaN,Numeric with tag,2749.0
157,220650,Total Protein,Total Protein,Labs,NaN,Numeric,2478.0
1794,226184,Estimated Protein Needs/Kg,Estimated Protein Needs/Kg,General,g/kg,Numeric,450.0
93,220454,Protein,Protein,Ingredients,grams,Ingredient,NaN
148,220612,ZC Reactive Protein (CRP),ZC Reaction Protein (CRP),Labs,NaN,Numeric,NaN
1667,225970,Beneprotein,Beneprotein,Nutrition - Enteral,grams,Solution,NaN
3514,229296,Vital High Protein (Full),Vital High Protein (Full),Nutrition - Enteral,mL,Solution,NaN
3713,229583,Beneprotein.,Beneprotein.,Nutrition - Supplements,mL,Solution,NaN


## `respiratory_support`

In [ ]:
resp_df = ItemFinder("Respiratory", col="category").candidate_table
resp_df

item label: Respiratory Rate; value instances: Max: 2355560.0, Min: 0.0, Mean: 20.459964592066456
item label: O2 saturation pulseoxymetry; value instances: Max: 9900000.0, Min: -272.0, Mean: 104.82763597187437
item label: O2 Delivery Device(s); value instances: ['Endotracheal tube', 'Nasal cannula', 'Tracheostomy tube', 'Aerosol-cool', 'Face tent', 'Trach mask ', 'High flow nasal cannula', 'Bipap mask ', 'High flow neb', 'Non-rebreather', 'CPAP mask ', 'Venti mask ', 'Other', 'Medium conc mask ', 'Oxymizer', 'T-piece', 'Ultrasonic neb', 'Vapomist']
item label: Inspired O2 Fraction; value instances: Max: 10050.0, Min: 0.0, Mean: 48.94540639505812
item label: PEEP set; value instances: Max: 8774580.0, Min: -5.0, Mean: 26.214116360816025
item label: Tidal Volume (observed); value instances: Max: 709461.0, Min: 0.0, Mean: 470.8951592275197
item label: Minute Volume; value instances: Max: 18918.4, Min: 0.0, Mean: 9.379572454568208
item label: Mean Airway Pressure; value instances: Max: 2577

,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances
28,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,6393762.0,"Max: 2355560.0, Min: 0.0, Mean: 20.45996459206..."
36,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,6324341.0,"Max: 9900000.0, Min: -272.0, Mean: 104.8276359..."
1946,226732,O2 Delivery Device(s),O2 Delivery Device(s),chartevents,Respiratory,NaN,Text,1536870.0,"['Endotracheal tube', 'Nasal cannula', 'Trache..."
384,223835,Inspired O2 Fraction,FiO2,chartevents,Respiratory,NaN,Numeric,849875.0,"Max: 10050.0, Min: 0.0, Mean: 48.94540639505812"
40,220339,PEEP set,PEEP set,chartevents,Respiratory,cmH2O,Numeric,642089.0,"Max: 8774580.0, Min: -5.0, Mean: 26.2141163608..."
...,...,...,...,...,...,...,...,...,...
3969,229992,Filters changed (Anes),Filters changed (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox
3970,229993,Vaporizer filled (Anes),Vaporizer filled (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox
3971,229994,Vaporizer checked (Anes),Vaporizer checked (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox
3972,229995,Amsorb changed (Anes),Amsorb changed (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox


In [ ]:
ItemFinder("Respiratory", col="category").candidate_table

### `peak_inspiratory_pressure_*`

In [42]:
ItemFinder("ipap").candidate_table

item label: BiPap Mask; value instances: ['Oral/Nasal', 'Total face', 'Nasal', 'Pillows', 'Not applicable']
item label: BiPap Mode; value instances: ['Spontaneous (S)', 'Spontaneous/Timed (S/T) (Back up)', 'Not applicable', 'Timed (T)']
item label: BiPap EPAP; value instances: Max: 136.0, Min: 0.0, Mean: 7.367685054219708
item label: BiPap IPAP; value instances: Max: 45.0, Min: 0.0, Mean: 13.283781379719311
item label: BiPap O2 Flow; value instances: Max: 100.0, Min: 0.0, Mean: 6.749544257646344
item label: BiPap bpm (S/T -Back up); value instances: Max: 22.0, Min: 0.0, Mean: 10.139130434782608


,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances
2284,227578,BiPap Mask,BiPap Mask,chartevents,Respiratory,NaN,Text,11315,"['Oral/Nasal', 'Total face', 'Nasal', 'Pillows..."
2283,227577,BiPap Mode,BiPap Mode,chartevents,Respiratory,NaN,Text,10648,"['Spontaneous (S)', 'Spontaneous/Timed (S/T) (..."
2285,227579,BiPap EPAP,BiPap EPAP,chartevents,Respiratory,cmH2O,Numeric,10605,"Max: 136.0, Min: 0.0, Mean: 7.367685054219708"
2286,227580,BiPap IPAP,BiPap IPAP,chartevents,Respiratory,cmH2O,Numeric,10118,"Max: 45.0, Min: 0.0, Mean: 13.283781379719311"
2288,227582,BiPap O2 Flow,BiPap O2 Flow,chartevents,Respiratory,L/min,Numeric,9874,"Max: 100.0, Min: 0.0, Mean: 6.749544257646344"
2287,227581,BiPap bpm (S/T -Back up),BiPap bpm (S/T -Back up),chartevents,Respiratory,bpm,Numeric,920,"Max: 22.0, Min: 0.0, Mean: 10.139130434782608"


### `tracheostomy`

In [ ]:
ItemFinder("Airway").candidate_table

item label: Mean Airway Pressure; value instances: Max: 257708.0, Min: -1.0, Mean: 10.914437783590904
item label: Airway Type; value instances: ['Oral', 'Nasal trumpet', 'Nasotracheal']
item label: Airway Location; value instances: ['Oral', 'Not applicable', 'L Nare', 'R Nare']
item label: Airway Size (mm); value instances: ['Not applicable', '8.0', '7.0', '9.0', '7.5']
item label: Airway problems; value instances: ['P > 30cm/H2O', 'Positional leak around cuff', 'No leak with cuff down', 'Cuff / valve leak']
item label: Unable to complete Airway Assessment (Intubation); value instances: []


,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances
806,224697,Mean Airway Pressure,Mean Airway Pressure,chartevents,Respiratory,cmH2O,Numeric,600557.0,"Max: 257708.0, Min: -1.0, Mean: 10.91443778359..."
385,223836,Airway Type,Airway Type,chartevents,Respiratory,NaN,Text,33002.0,"['Oral', 'Nasal trumpet', 'Nasotracheal']"
2280,227567,Airway Location,Airway Location,chartevents,Respiratory,NaN,Text,4686.0,"['Oral', 'Not applicable', 'L Nare', 'R Nare']"
2281,227568,Airway Size (mm),Airway Size (mm),chartevents,Respiratory,NaN,Text,3087.0,"['Not applicable', '8.0', '7.0', '9.0', '7.5']"
1992,226815,Airway problems,Airway problems,chartevents,Respiratory,NaN,Text,1739.0,"['P > 30cm/H2O', 'Positional leak around cuff'..."
1198,225292,Unable to complete Airway Assessment (Intubation),Unable to complete Airway (Intubation),chartevents,Intubation,NaN,Checkbox,NaN,Checkbox
2618,228066,Unable to complete Airway Assessment (Intubation),Unable to complete Assessment (Intubation),chartevents,Intubation,NaN,Text,NaN,[]


In [ ]:
ItemFinder("trach").candidate_table

item label: Trach Tube Type; value instances: ['Perc Trach ', 'Cuffed', 'Standard', 'Inner Cannula', 'TTS (Tight to Shaft)', 'Adjustable Neck Flange', 'Extra Length', 'Uncuffed', 'Wire Reinforced', 'Not applicable', 'Foam Cuff', 'Fenestrated']
item label: Trach Tube Manufacturer; value instances: ['Portex', 'Bivona', 'Shiley', 'Not applicable', 'Boston Med']
item label: Trach Tube Size (I.D.); value instances: ['8.0mm', '7.0mm', '6.0mm', '9.0mm', 'Not applicable', '5.0mm', '10.0mm', '4.0mm']
item label: ZTrach Care; value instances: []
item label: Tracheal Confirmation (Intubation); value instances: []


,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances
886,224829,Trach Tube Type,Trach Tube Type,chartevents,Respiratory,NaN,Text,178441.0,"['Perc Trach ', 'Cuffed', 'Standard', 'Inner C..."
887,224830,Trach Tube Manufacturer,Trach Tube Manufacturer,chartevents,Respiratory,NaN,Text,120298.0,"['Portex', 'Bivona', 'Shiley', 'Not applicable..."
888,224831,Trach Tube Size (I.D.),Trach Tube Size (I.D.),chartevents,Respiratory,NaN,Text,119572.0,"['8.0mm', '7.0mm', '6.0mm', '9.0mm', 'Not appl..."
2136,227130,Trach Care,Trach Care,chartevents,Respiratory,NaN,Checkbox,43499.0,Checkbox
1306,225448,Percutaneous Tracheostomy,Percutaneous Tracheostomy,procedureevents,1-Intubation/Extubation,NaN,Processes,590.0,Processes
1808,226237,Open Tracheostomy,Open Tracheostomy,procedureevents,1-Intubation/Extubation,NaN,Processes,24.0,Processes
912,224864,ZTrach Care,ZTrach Care,chartevents,Respiratory,NaN,Text,NaN,[]
1383,225590,Tracheal Confirmation (Intubation),Tracheal Confirmation (Intubation),chartevents,Intubation,NaN,Text,NaN,[]


In [ ]:
item_id_to_events(227130)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
142713,10004401,22869003,38292466,16647.0,2144-04-07 15:41:00,2144-04-07 15:41:00,227130,1,1.0,None,0.0
143071,10004401,22869003,38292466,16647.0,2144-04-08 12:00:00,2144-04-08 18:59:00,227130,1,1.0,None,0.0
143385,10004401,22869003,38292466,16681.0,2144-04-05 20:22:00,2144-04-05 20:23:00,227130,1,1.0,None,0.0
145916,10004401,22869003,38292466,96601.0,2144-04-07 21:12:00,2144-04-07 21:12:00,227130,1,1.0,None,0.0
146563,10004401,22869003,38292466,96601.0,2144-04-08 22:28:00,2144-04-08 22:28:00,227130,1,1.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313425095,19995258,26871572,38607853,29441.0,2130-06-12 02:00:00,2130-06-12 02:02:00,227130,0,0.0,None,0.0
313442191,19995595,21784060,34670930,48419.0,2126-10-24 04:17:00,2126-10-24 04:21:00,227130,0,0.0,None,0.0
313445749,19995595,21784060,34670930,68716.0,2126-10-21 20:00:00,2126-10-21 23:33:00,227130,0,0.0,None,0.0
313445852,19995595,21784060,34670930,68716.0,2126-10-21 23:00:00,2126-10-21 23:33:00,227130,0,0.0,None,0.0


In [ ]:
item_id_to_events(225448, procedureevents)

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
311,10004401,29988601,32773003,11464.0,2144-01-31 14:30:00,2144-01-31 14:31:00,2144-01-31 17:05:00,225448,1.0,NaN,...,3753247,3753247,Procedures,Task,76.0,0,0,FinishedRunning,1.0,0
3921,10057482,25416257,38980177,13374.0,2145-04-07 16:16:00,2145-04-07 16:17:00,2145-04-07 16:30:00,225448,1.0,NaN,...,6779900,6779900,Procedures,Task,62.0,0,0,FinishedRunning,1.0,0
3949,10057482,25416257,38980177,40517.0,2145-04-07 15:30:00,2145-04-07 15:31:00,2145-04-07 17:54:00,225448,1.0,NaN,...,6373213,6373213,Procedures,Task,62.0,0,0,FinishedRunning,1.0,0
4927,10078115,28052678,36265401,98628.0,2147-12-09 11:40:00,2147-12-09 11:41:00,2147-12-09 12:55:00,225448,1.0,NaN,...,2123399,2123399,Procedures,Task,69.4,0,0,FinishedRunning,1.0,0
7480,10123036,29540743,30112389,95471.0,2168-06-15 16:06:00,2168-06-15 16:07:00,2168-06-15 17:06:00,225448,1.0,NaN,...,5029949,5029949,Procedures,Task,95.4,0,0,FinishedRunning,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693567,19969918,29544887,35223685,82842.0,2186-02-04 14:30:00,2186-02-04 14:31:00,2186-02-04 15:13:00,225448,1.0,NaN,...,1444170,1444170,Procedures,Task,68.0,0,0,FinishedRunning,1.0,0
693602,19970265,24514478,38356273,12302.0,2115-04-03 10:00:00,2115-04-03 10:01:00,2115-04-03 13:12:00,225448,1.0,NaN,...,9268772,9268772,Procedures,Task,67.9,0,0,FinishedRunning,1.0,0
693755,19970491,25338284,30244200,17526.0,2129-06-01 11:24:00,2129-06-01 11:25:00,2129-06-01 11:24:00,225448,1.0,NaN,...,8281334,8281334,Procedures,Task,80.0,0,0,FinishedRunning,1.0,0
693975,19971771,26230047,38668070,44464.0,2117-05-09 15:24:00,2117-05-09 15:25:00,2117-05-09 15:33:00,225448,1.0,NaN,...,3839490,3839490,Procedures,Task,90.0,0,0,FinishedRunning,1.0,0


### `lpm_set`

In [ ]:
ItemFinder("liter").candidate_table

item label: Non Invasive Blood Pressure systolic; value instances: Max: 1025100.0, Min: -69.0, Mean: 120.67245107644807
item label: Non Invasive Blood Pressure diastolic; value instances: Max: 114109.0, Min: -2.0, Mean: 66.418738289522
item label: Non Invasive Blood Pressure mean; value instances: Max: 8888890.0, Min: -22767.0, Mean: 84.59097686297358
item label: Arterial Blood Pressure mean; value instances: Max: 930000.0, Min: -135.0, Mean: 80.14749761559462
item label: Arterial Blood Pressure systolic; value instances: Max: 95119.0, Min: -94.0, Mean: 119.34178904143593
item label: Arterial Blood Pressure diastolic; value instances: Max: 114100.0, Min: -40.0, Mean: 60.06301600912156
item label: Pressure Reducing Device; value instances: ['Mepilex to coccyx', 'Waffle Boots', 'Multipodis Boots', 'Foam Wedge', 'Heel/Elbow Pads', 'Seat Cushion', 'Sheepskin', 'Foam Pad', 'Footboard/Cradle']
item label: Central Venous Pressure; value instances: Max: 7785.0, Min: -41.0, Mean: 14.70150991838

KeyboardInterrupt: 

### `minute_vent_obs`

In [ ]:
vent_df = ItemFinder("vent").candidate_table
vent_df

item label: Altered Respiratory Status NCP - Interventions; value instances: ['Assess RR pattern, oxygenation, and auscultation', 'HOB > 30 degrees', 'Oral care every 2-4 hours, remove all debris', 'Aspiration precautions', 'Repositioning for optimizing respiratory status', 'Every 2 hour turns', 'Suction PRN', 'Encourage cough and deep breathing', 'Antibiotic therapy for respiratory infection', 'Intubation', 'Culture sputum and blood for new fever', 'Restraints for safety with every 24 hour physician order', 'Nebulizer treatments', 'OOB to chair', 'Daily RSBI', 'PEG or NGT placement for aspiration risk', 'SBT', 'Diuresis', 'Daily wake-up', 'Chest PT', 'Incentive spirometry', 'Ventilator support increased', 'Specialty bed', 'Non-invasive ventilatory support', 'Tracheostomy care and maintenance', 'Extubation', 'Interventions for angioedema', 'Tracheostomy placement', 'Maintain chest tube(s)', 'Other', '---']
item label: Impaired Tissue Perfusion NCP - Interventions; value instances: ['As

,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances
3283,228905,Altered Respiratory Status NCP - Interventions,ARS NCP - Interventions,chartevents,Care Plans,NaN,Text,1874646.0,"['Assess RR pattern, oxygenation, and ausculta..."
3291,228928,Impaired Tissue Perfusion NCP - Interventions,ITP NCP - Interventions,chartevents,Care Plans,NaN,Text,1472246.0,"['Assess indices of perfusion (e.g. UO, hemody..."
3316,228988,Safety Restraints NCP - Interventions,Safety Restraints NCP - Interventions,chartevents,Care Plans,NaN,Text,1391021.0,['Obtain M.D. order for nonviolent restraints ...
3299,228947,Infection NCP - Interventions,Infection NCP - Interventions,chartevents,Care Plans,NaN,Text,1075834.0,"['Monitor and report VS, labs, etc. of concern..."
3280,228898,Impaired Fluid Balance NCP - Interventions,IFB NCP - Interventions,chartevents,Care Plans,NaN,Text,890590.0,['Monitor vital signs and hemodynamic pressure...
...,...,...,...,...,...,...,...,...,...
3926,229869,Pressure Injury Prevention Interventions,Pressure Injury Prevention Interventions,chartevents,Skin - Assessment,NaN,Text,NaN,[]
3954,229899,Left Ventricular Pressure Signal - Systolic,Left Ventricular Pressure Signal - Systolic,chartevents,Impella,mmHg,Numeric,NaN,"Max: nan, Min: nan, Mean: nan"
3955,229900,Left Ventricular Pressure Signal - Diastolic,Left Ventricular Pressure Signal - Diastolic,chartevents,Impella,mmHg,Numeric,NaN,"Max: nan, Min: nan, Mean: nan"
3958,229977,COVID NCP - Interventions,COVID NCP - Interventions,chartevents,Care Plans,NaN,Text,NaN,[]
